In [1]:
import pandas as pd
import numpy as np
csv= pd.read_csv(r'C:\Users\HP\Desktop\MLWARE\train.csv',index_col = 0)

csv = csv.dropna()
csv.head()

,title,domain,over_18,time_submitted,submitter_id,engaging
id,,,,,,
66683,Coronation Street spoilers: David Platt rape t...,youtube.com,0,1521358296,15336.0,0
42713,Japan lawmaker calls sexual harassment whistle...,japantoday.com,0,1524749734,4977.0,1
46874,16 Completely Improper Halloween Costumes For ...,rafaladefrumusete.info,0,1521811040,2407.0,0
36489,Humane prison to bring Greenland's most danger...,edition.cnn.com,0,1521305721,729.0,1
52038,Global Offshore Drilling Fluids Market: Indust...,satenews.com,0,1521246642,13169.0,0


In [2]:
title=csv['title']
title.head()

id
66683    Coronation Street spoilers: David Platt rape t...
42713    Japan lawmaker calls sexual harassment whistle...
46874    16 Completely Improper Halloween Costumes For ...
36489    Humane prison to bring Greenland's most danger...
52038    Global Offshore Drilling Fluids Market: Indust...
Name: title, dtype: object

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# Construct a bag of words matrix.
# This will lowercase everything, and ignore all punctuation by default.
# It will also remove stop words.
vectorizer = CountVectorizer(lowercase=True, stop_words="english")

matrix = vectorizer.fit_transform(title)

print(matrix.todense())

#  added the url of the submission to the end of the headline so we can take it into account.
csv['full_test'] = csv["title"] + " " + csv["domain"]
full_matrix = vectorizer.fit_transform(csv["full_test"])
print(full_matrix.shape)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(49677, 53028)


In [4]:
v=np.random.randint(full_matrix.shape[0], size=34000)
full_matrix_new=full_matrix[v, :]
full_matrix_new.shape

(34000, 53028)

In [5]:
import re
import numpy
# Our list of functions to apply.
transform_functions = [
    lambda x: len(x),
    lambda x: x.count(" "),
    lambda x: x.count("."),
    lambda x: x.count("!"),
    lambda x: x.count("?"),
    lambda x: len(x) / (x.count(" ") + 1),
    lambda x: x.count(" ") / (x.count(".") + 1),
    lambda x: len(re.findall("\d", x)),
    lambda x: len(re.findall("[A-Z]", x)),
]

# Apply each function and put the results into a list.
columns = []
for func in transform_functions:
    columns.append(csv["title"].apply(func))
    
# Convert the meta features to a numpy array.
meta = numpy.asarray(columns).T

In [6]:
submission_time = csv["time_submitted"]
y=submission_time.mean()
submission_time=(submission_time-y)/1000000


submission_time= numpy.asarray(submission_time).T

In [7]:
submitter_id=csv["submitter_id"]
x=submitter_id.mean()
submitter_id=(submitter_id-x)/1000
submitter_id= numpy.asarray(submitter_id).T

In [8]:
meta.shape

(49677, 9)

In [9]:
features = numpy.column_stack([submission_time[0:34000][:],submitter_id[0:34000][:], meta[0:34000][:], full_matrix_new.todense()])

In [10]:
csv_test= pd.read_csv(r'C:\Users\HP\Desktop\MLWARE\test.csv',index_col = 0)
csv_test = csv_test.dropna()
title2=csv_test['title']


# Construct a bag of words matrix.
# This will lowercase everything, and ignore all punctuation by default.
# It will also remove stop words.


matrix2 = vectorizer.transform(title2)

print(matrix2.todense())


csv_test['full_test'] = csv_test["title"] + " " + csv_test["domain"]
full_matrix2 = vectorizer.transform(csv_test["full_test"])
print(full_matrix2.shape)

columns2 = []
for func in transform_functions:
    columns2.append(csv_test["title"].apply(func))
    
# Convert the meta features to a numpy array.
meta2 = numpy.asarray(columns2).T

submission_time2 = csv_test["time_submitted"]
z=submission_time2.mean()
submission_time2=(submission_time2-z)/1000000


submission_time2= numpy.asarray(submission_time2).T

submitter_id2=csv_test["submitter_id"]
u=submitter_id2.mean()
submitter_id2=(submitter_id2-u)/1000
submitter_id2= numpy.asarray(submitter_id2).T

features2 = numpy.column_stack([submission_time2,submitter_id2, meta2, full_matrix2.todense()])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(16743, 53028)


In [11]:
col2=csv.iloc[v,5]

In [12]:

from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(features, col2)
predicted_classes = model.predict(features2)

In [13]:
d={'id':list(csv_test.index),'engaging':predicted_classes}
df=pd.DataFrame(data=d)
df.head()

,id,engaging
0,32722,1
1,30618,0
2,9627,1
3,13762,0
4,22905,0


In [15]:
df.to_csv(r'C:\Users\HP\Desktop\MLWARE\Book1.csv', index=False, header=False)